In [73]:
# importing libraries
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nexinfosys import NISClient, display_visjs_jupyterlab
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from nexinfosys import NISClient, display_visjs_jupyterlab
import io
import networkx as nx
import csv
import qgrid

In [74]:
def write_csv(output_csv,name): #eg flow_graph_solution.csv
    with open(name,'wb') as file:
        for line in output_csv:
            file.write(line)
    return name

In [75]:
# Input Nis Format File
fname =  'howTo Clone CloneScale.xlsx'
#fname = '/home/paula/Documentos/JN/DB_Biofuel_Europa_Diagnostic ModeV6_formatNIS_2016Escenarios.xlsx'

In [76]:
!ls

EDA-Notebook.ipynb
flow_graph_solution2.csv
flow_graph_solution_clean.csv
flow_graph_solution.csv
howTo Clone CloneScale.xlsx
nexinfosys_python-notebook-script_Widgets_Basic_explotations.ipynb
README.md


### Connect to Nis Client

In [77]:
#c = NISClient("https://one.nis.magic-nexus.eu/nis_api")
#To local:
#c = NISClient("http://localhost:5000/nis_api")
c = NISClient("http://172.17.0.1:5000/nis_api")
# Login, open session, load a workbook (which is in Nextcloud), submit (execute!)
c.login("test_user")
#print("Logged in")

True

In [78]:
c.open_session()
#print("Session opened")

True

In [79]:
n = c.load_workbook(fname)
print("N worksheets: "+str(n))

N worksheets: 7


In [80]:
r = c.submit()

In [81]:
any_error = False
if len(r) > 0:
    for i in r:
        if i["type"] == 3:
            any_error = True
            print(str(i))
            

In [82]:
if not any_error:
    # Obtain available datasets
    r = c.query_available_datasets()
    if len(r) > 0:
        results = {}
        for ds in r:
            results[ds["name"]] = {d["format"].lower(): d["url"] for d in ds["formats"]}


In [83]:
#See Warnings
#pd.DataFrame(r)

### Available Resuts

In [84]:
pd.DataFrame(c.query_available_datasets())

,description,formats,name,type
0,"Parameter values per Scenario [4 rows, 12 cell...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",params,dataset
1,"Flow Graph Solver - Interfaces [108 rows, 1944...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution,dataset
2,"Flow Graph Solver Edges - Interfaces [42 rows,...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution_edges,dataset
3,"Graph of Interfaces, Quantities; Scales and Ex...","[{'format': 'VisJS', 'url': '/nis_api/isession...",interfaces_graph,graph
4,Processors and exchanges graph,"[{'format': 'VisJS', 'url': '/nis_api/isession...",processors_graph,graph
5,Dictionary of Sankey Graph for every scenario ...,"[{'format': 'JSON', 'url': '/nis_api/isession/...",Sankey_Graph,Graph
6,Processors,"[{'format': 'GeoJSON', 'url': '/nis_api/isessi...",processors_geolayer,geolayer
7,Model,"[{'format': 'JSON', 'url': '/nis_api/isession/...",Model,model
8,OWL ontology,"[{'format': 'OWL', 'url': '/nis_api/isession/r...",Ontology,ontology
9,Python script,"[{'format': 'Python', 'url': '/nis_api/isessio...",Python script,script


In [85]:
#TODO widget para elegir output y descagarlo

## Processors Graph

In [86]:
r = c.download_results([(results["processors_graph"]["visjs"])])
visjs_data2 = r[0].decode("utf-8")
un2 = None

#unique_name = display_visjs_jupyterlab(visjs_data, 800, unique_name)
un2 = display_visjs_jupyterlab(visjs_data2, 1200, un2)


<IPython.core.display.Javascript object>

## Exploring Flow Graph Solution

In [87]:
flow_graph_solution = c.download_results([(results["flow_graph_solution"]['csv'])])
write_csv(flow_graph_solution,"flow_graph_solution.csv")
raw = pd.read_csv("flow_graph_solution.csv")

In [89]:
qgrid_widget = qgrid.show_grid(raw,show_toolbar= True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [90]:
raw.columns

Index(['Scenario', 'Period', 'Scope', 'Conflict_Partof', 'Conflict_Itype',
       'Processor', 'Interface', 'Orientation', 'RoegenType', 'Value',
       'Computed', 'Observer', 'Expression', 'Unit', 'Level', 'System',
       'Subsystem', 'Sphere'],
      dtype='object')

### Data Clonflict

In [124]:
def ConflictStudy(df,conflictType): 
    if any(raw[conflictType] =='Dismissed'):
        Taken = df[df[conflictType]=='Taken'].drop([conflictType], axis= 1)
        Dismissed = df[df[conflictType]=='Dismissed'].drop([conflictType], axis= 1)
        Taken = Taken.rename(columns={'Value':'Value_Taken'})
        Dismissed = Dismissed.rename(columns={'Value':'Value_Dismissed'})
        Conflict = pd.merge(Taken,Dismissed, how = 'outer')
        Conflict['diff'] = Conflict['Value_Taken'] - Conflict['Value_Dismissed']
        return Conflict['diff']
    else:
        print('No Conflicts found')

#### Part Of Conflict

In [116]:
ConflictItype = ConflictStudy(raw,'Conflict_Itype')

No Conflicts found


In [125]:
ConflictStudy(raw,'Conflict_Partof')

  Scenario  Period     Scope Conflict_Itype               Processor Interface  \
0  default    2020  External             No  France.crop_production     wheat   
1  default    2020  Internal             No   Spain.crop_production     wheat   
2  default    2020     Total             No  France.crop_production     wheat   
3  default    2020     Total             No   Spain.crop_production     wheat   
4  default    2020  External             No  France.crop_production     wheat   
5  default    2020  Internal             No   Spain.crop_production     wheat   
6  default    2020     Total             No  France.crop_production     wheat   
7  default    2020     Total             No   Spain.crop_production     wheat   

  Orientation RoegenType  Value_Taken Computed  Observer  \
0      Output       Flow        100.0       No       NaN   
1      Output       Flow        100.0       No       NaN   
2      Output       Flow        100.0       No       NaN   
3      Output       Flow      

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
Name: diff, dtype: float64

### Flow graph solution Study

#### Absolute Values Mode

In [59]:
raw = raw[(raw['Conflict_Partof'] != 'Dissmissed')&(raw['Conflict_Itype']!='Dismissed')]
#Simplified dataFrame for accounting exploration
df = raw.drop(columns= ['Conflict_Partof','Conflict_Itype', 'Computed','Observer','Expression'])
# los observers cambian según el escenario entonces en realidad daría igual si dejarlo o no en el dataframe??

Inputs = df[df['Orientation']=='Input']
Outputs = df[df['Orientation'] == 'Output']

Inputs =Inputs.rename(columns= {'Value': 'Value_Input'})
Outputs = Outputs.rename(columns={'Value':'Value_Output'})

Inputs = Inputs.drop(columns = "Orientation")
Outputs = Outputs.drop(columns = "Orientation")

data = pd.merge(Inputs,Outputs, how = 'outer')

data['Value'] = abs(data['Value_Input'].fillna(0)-data['Value_Output'].fillna(0))

In [60]:
data.to_csv('flow_graph_solution_clean.csv')


In [61]:
# Ver porqué no me funciona esto
qgrid_widget = qgrid.show_grid(data,show_toolbar= True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Dar la opción de ver input y output o de verlo en total en la gráfica de barras poner el input como +y aoutput como negativo?

### BarChart by Level

In [62]:
fig = go.FigureWidget( layout=go.Layout())
@interact(Level = list(set(data['Level'])), 
          Scenario = list(set(data['Scenario'])),
          Period = list(set(data['Period'])),
          Scope = list(set(data['Scope'])),
          Interface = list(set(data['Interface']))
         )
def Bar_chart_Level(Level,Scenario,Period,Scope,Interface):
    data = pd.read_csv("flow_graph_solution_clean.csv")
    data = data[(data['Scenario'] == Scenario)&(data['Level'] == Level)&(data['Period'] == Period)&(data['Scope'] == Scope)&(data['Interface'] == Interface)]    
    y_pos =np.arange(len(data.Processor))
    plt.bar(y_pos,data['Value'])
    plt.xticks(y_pos,data.Processor, rotation = 90)
    plt.show()

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)   
    
    

interactive(children=(Dropdown(description='Level', options=('n-1', 'n', 'n-2', 'n-4', 'n-3'), value='n-1'), D…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [63]:
fig = go.FigureWidget( layout=go.Layout())
@interact(Scope = list(set(data['Scope'])),
          Interface = list(set(data['Interface'])),
          Processor = list(set(data['Processor'])),
          Period = list(set(data['Period']))
         )
def Bar_chart_scenario(Processor,Period,Scope,Interface):
    data = pd.read_csv("flow_graph_solution_clean.csv")
    data = data[(data['Processor'] == Processor)&(data['Period'] == Period)&(data['Scope'] == Scope)&(data['Interface'] == Interface)]    
    y_pos =np.arange(len(data.Scenario))
    plt.bar(y_pos,data.Value)
    plt.xticks(y_pos,data.Scenario, rotation = 90)
    plt.show()

interactive(children=(Dropdown(description='Processor', options=('G.Convencional.GH', 'P', 'P.Convencional.GH.…

In [65]:
fig = go.FigureWidget( layout=go.Layout())
@interact(Scope = list(set(data['Scope'])),
          Interface = list(set(data['Interface'])),
          Scenario = list(set(data['Scenario'])),
          Level = list(set(data['Level']))
         )
def Time_Interface(Scope,Interface,Level,Scenario):
    data = pd.read_csv("flow_graph_solution2.csv")
    data = data[(data['Level'] == Level)&(data['Scope'] == Scope)&(data['Interface'] == Interface)&(data['Scenario']== Scenario)]  
    Processors= list(set(data['Processor']))
    Unit = list(set(data['Unit']))  
    datatempspread =  data[['Processor','Value','Period']].pivot(index = 'Period',columns='Processor',values='Value')
    datatempspread = datatempspread.reset_index()
    fig, ax = plt.subplots()
    for nm in Processors:
        ax.plot(datatempspread['Period'],datatempspread[nm], label=nm)
        # Set x-ticks to yearly interval, adjust y-axis limits, add legend and labels   
        ax.legend()
        ax.set_ylabel(Interface[0]+' ' + Unit[0])
        ax.set_title(Level);
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.show()

interactive(children=(Dropdown(description='Scope', options=('Total', 'Internal'), value='Total'), Dropdown(de…

In [67]:
c.close_session()

True

In [68]:
c.logout()

True